In [1]:
import pandas as pd
import os

import sys
sys.path.append('/mnt/0A2AAC152AABFBB7/CGE/paper-fetcher')
from paper_fetcher.country import CountryRecoder, Apostrophe

from paper_fetcher.utils import recover_columns_names

from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer

In [2]:
FOLDER = '/mnt/0A2AAC152AABFBB7/data/PubMedPapers/'
df_all = pd.read_csv(
    os.path.join(FOLDER, 'publications_affi_cleaned.csv')
)
df_all['country'] = df_all['country'].astype(str)

In [3]:
usa_locus = [
    "USA", "United States", "US", "Alabama", "California", "Albuquerque", "Ames IA", "Amherst MA", "Jackson MS",
    "Jacksonville FL", "Ann Arbor", "Albert Einstein College", "American Cancer Society", "Atlanta", "Massachusetts",
    "Boston", "Medford MA", "Milwaukee WI", "Arizona", "Baltimore", "Berkeley CA", "Bethesda MD", "Birmingham AL",
    "Boulder CO", "Aurora CO", "Cambridge MA", "Harvard MA", "Brookline MA", "Brownsville TX", "Bronx NY", "New Haven CT",
    "New Brunswick NJ", "Buffalo NY", "Burlington VT", "Califirnia", "New York", "Norwood MA", "Oakland", "Oregon",
    "Palo Alto CA", "San Francisco CA", "Chapel Hill NC", "Charleston SC", "Charlestown MA", "Charlotte NC", "Dallas",
    "Pennsylvania PA", "Pennsylvania", "Phoenix AZ", "Charlottesville VA", "Chicago", "Cincinnati", "Claremont CA",
    "Cleveland OH", "Collegeville PA", "Columbia SC", "Columbia University", "Providence RI", "Chevy Chase MD",
    "Puerto Rico", "Redwood City CA", "Triangle Park NC", "Columbus OH", "Corona CA", "Philadelphia", "Rochester MN",
    "Rockville MD", "Sacramento CA", "Salt Lake City", "San Antonio TX", "Seattle", "Rhode Island", "Brooklyn College",
    "Brigham and Womens Hospital", "Calico", "Sioux Center IA", "CedarsSinai", "Los Angeles", "Colorado", "Danville PA",
    "Davis CA", "Decatur GA", "Denver", "Stanford University", "Stanford Medical School", "Delaware", "West Virginia",
    "Louisville KY", "Brown University", "Blacksburg VA", "District of Columbia", "Downers Grove IL", "Duarte CA",
    "Duke University", "Tennessee", "Texas", "East Lansing", "Emory University", "Englewood CO", "Evanston IL",
    "George Washington University", "Icahn School", "Durham NC", "Farmington CT", "Florida", "Fort Collins CO",
    "Fort Worth TX", "Framingham", "Frederick MD", "Fremont CA", "Gainesville", "Gaithersburg MD", "Grand Rapids MI",
    "Glen Echo MD", "Hagerstown MD", "Hanover NH", "Harrison NJ", "Hartford CT", "Hawaii", "Mississippi", "Michigan",
    "Miami", "Utah", "Honolulu", "Houston", "Illinois", "Iowa", "Irvine CA", "Ithaca NY", "Johns Hopkins", "Kalamazoo MI",
    "Kansas", "Kentucky", "Prussia PA", "La Jolla", "Lancaster PA", "Las Vegas NV", "Lewisburg PA", "Lexington KY",
    "Lisle IL"
]

In [4]:
esp_locus = ["Spain", "España",  "Barcelona", "Madrid", "Ramón y Cajal", "Asturias", "Pamplona", "Granada"]
fra_locus = ["France", "Marseille", "Bordeaux", "Paris", "Inserm", "Centre de Recherche des Cordeliers", "Créteil",
             "Dijon", "Lille", "Nantes", "Montpellier", "la Santé"]
aus_locus = ["Australia", "Melbourne", "Perth", "Cancer Council Victoria", "Curtin University"]
den_locus = ["Denmark", "Aarhus", "Danish", "Danmark", "Faroe Islands", "Greenland", "Herlev and Gentofte Hospital"]
uk_locus  = ["United Kingdom", "UK", "Aberdeen", "Addenbrookes Hospital", "N Ireland", "Scotland", "Belfast", "Bristol",
            "Cardiff", "London", "NHS Trust", "NHS trust", "British", "Edinburgh", "Glasgow", "NHS Foundation Trust",
            "John Radcliffe Hospital", "Lancaster University", "Leeds", "Leicester", "Liverpool"]
civ_locus = ["Côte dIvoire"]
uae_locus = ["UAE", "United Arab Emirates", "Abu Dhabi"]
nig_locus = ["Nigeria", "Abuja", "Lagos"]
chn_locus = ["China", "Beijing", "CHINA", "Chinese", "Central South University", "Changping", "Chengdu", "Dongfang",
             "Xian Jiaotong", "Tongxiang", "Fudan University", "Guangzhou", "Guangdong", "Harbin", "Heilongjiang",
             "Furong", "Guangxi", "Hong Kong", "Hongqiao", "Dongguan", "Hunan", "Jiangsu", "Jinan", "Zhengzhou",
             "Zhongshan", "Liaoning", "Liuzhou"]
eth_locus = ["Ethiopia"]
jpn_locus = ["Japan", "Tokyo", "Osaka", "Chiba University", "Tokushima", "Tohoku", "Fukuoka", "Hiroshima",
             "Iwate Medical University", "Kanagawa"]
swe_locus = ["Sweden", "Stockholm", "Malmö", "Lund", "Umeå", "Karolinska Institute", "Linköping"]
nth_locus = ["Netherlands", "Amsterdam", "EMGO", "Vrije Universiteit", "Leiden"]
ger_locus = ["Germany", "Heidelberg", "Augsburg", "Berlin", "Munich", "Deutschland", "Essen", "Freiburg", "Fraunhofer",
             "German", "Greifswald", "Helmholtz"]
aut_locus = ["Austria", "Graz", "Innsbruck"]
tur_locus = ["Turkey"]
gre_locus = ["Greece"]
ita_locus = ["Italy", "Ragusa","Sassari", "Rome", "Bergamo", "Roma", "DIBRIS", "Torino", "Florence", "Fondazione IRCCS",
             "ITALY", "Italia", "Itlay"]
col_locus = ["Colombia", "Bogotá", "Bogota"]
rus_locus = ["Russia", "Moscow"]
can_locus = ["Canada", "British Columbia", "Ontario", "Québec", "Quebec", "Toronto"]
nor_locus = ["Norway", "Oslo", "Bergen"]
bel_locus = ["Belgium", "Leuven"]
nz_locus  = ["New Zealand", "Auckland"]
fin_locus = ["Finland", "Oulu", "FINLAND", "Finlan", "Helsinki", "Kuopio"]
isr_locus = ["Israel"]
arg_locus = ["Argentina", "Buenos Aires"]
per_locus = ["Perú", "Peru"]
skr_locus = ["South Korea", "Republic of Korea", "Busan", "Seoul", "Cheongjusi", "Chungnam", "Daegu", "Daejeon", "Goyangsi",
             "Gwangju", "Gyeonggido", "Ulsan", "Hwasun", "Korea University"]
bra_locus = ["Brazil", "Brasil"]
chi_locus = ["Chile", "Talca"]
ind_locus = ["India"]
hun_locus = ["Hungary", "Budapest"]
tai_locus = ["Taiwan", "Taipei", "Chung Shan Medical", "Taichung", "Hualien", "Kaohsiung"]
ksa_locus = ["Saudi Arabia", "Riyadh"]
por_locus = ["Portugal", "Aveiro", "Coimbra"]
swi_locus = ["Switzerland", "Botnar Research Centre", "Zurich"]
egy_locus = ["Egypt"]
bru_locus = ["Brunei"]
mal_locus = ["Mali"]
tha_locus = ["Thailand"]
bar_locus = ["Barbados"]
cyp_locus = ["Cyprus"]
sin_locus = ["Singapore"]
ina_locus = ["Indonesia"]
bos_locus = ["Bosnia and Herzegovina"]
bah_locus = ["Bahrain"]
cam_locus = ["Cameroon", "Cameroun"]
rsa_locus = ["South Africa"]
phi_locus = ["Philippines"]
ban_locus = ["Bangladesh", "Dhaka"]
qat_locus = ["Qatar", "Doha"]
ire_locus = ["Ireland", "Dublin"]
mex_locus = ["Mexico", "México"]
est_locus = ["Estonia"]
slk_locus = ["Slovak"]
cz_locus  = ["Czech", "Prague"]
gha_locus = ["Ghana"]
ice_locus = ["Iceland", "Kopavogur"]
slv_locus = ["Slovenia"]
irn_locus = ["Iran"]
pak_locus = ["Pakistan"]
jam_locus = ["Jamaica"]
pol_locus = ["Poland"]

In [5]:
clean_pipe = Pipeline([
    ('apo', Apostrophe().set_output(transform="pandas")),
    ("esp", CountryRecoder(country="Spain",         locus=esp_locus).set_output(transform="pandas")),
    ("fra", CountryRecoder(country="France",        locus=fra_locus).set_output(transform="pandas")),
    ("aus", CountryRecoder(country="Australia",     locus=aus_locus).set_output(transform="pandas")),
    ("den", CountryRecoder(country="Denmark",       locus=den_locus).set_output(transform="pandas")),
    ("uk" , CountryRecoder(country="UK",            locus=uk_locus).set_output(transform="pandas")),
    ("civ", CountryRecoder(country="Côte d'Ivoire", locus=civ_locus).set_output(transform="pandas")),
    ("uae", CountryRecoder(country="UAE",           locus=uae_locus).set_output(transform="pandas")),
    ("nig", CountryRecoder(country="Nigeria",       locus=nig_locus).set_output(transform="pandas")),
    ("chn", CountryRecoder(country="China",         locus=chn_locus).set_output(transform="pandas")),
    ("eth", CountryRecoder(country="Ethiopia",      locus=eth_locus).set_output(transform="pandas")),
    ("jpn", CountryRecoder(country="Japan",         locus=jpn_locus).set_output(transform="pandas")),
    ("usa", CountryRecoder(country="USA",           locus=usa_locus).set_output(transform="pandas")),
    ("swe", CountryRecoder(country="Sweden",        locus=swe_locus).set_output(transform="pandas")),
    ("nth", CountryRecoder(country="Netherlands",   locus=nth_locus).set_output(transform="pandas")),
    ("ger", CountryRecoder(country="Germany",       locus=ger_locus).set_output(transform="pandas")),
    ("aut", CountryRecoder(country="Austria",       locus=aut_locus).set_output(transform="pandas")),
    ("tur", CountryRecoder(country="Turkey",        locus=tur_locus).set_output(transform="pandas")),
    ("gre", CountryRecoder(country="Greece",        locus=gre_locus).set_output(transform="pandas")),
    ("ita", CountryRecoder(country="Italy",         locus=ita_locus).set_output(transform="pandas")),
    ("col", CountryRecoder(country="Colombia",      locus=col_locus).set_output(transform="pandas")),
    ("rus", CountryRecoder(country="Russia",        locus=rus_locus).set_output(transform="pandas")),
    ("can", CountryRecoder(country="Canada",        locus=can_locus).set_output(transform="pandas")),
    ("nor", CountryRecoder(country="Norway",        locus=nor_locus).set_output(transform="pandas")),
    ("bel", CountryRecoder(country="Belgium",       locus=bel_locus).set_output(transform="pandas")),
    ("nz",  CountryRecoder(country="New Zealand",   locus=nz_locus).set_output(transform="pandas")),
    ("fin", CountryRecoder(country="Finland",       locus=fin_locus).set_output(transform="pandas")),
    ("isr", CountryRecoder(country="Israel",        locus=isr_locus).set_output(transform="pandas")),
    ("arg", CountryRecoder(country="Argentina",     locus=arg_locus).set_output(transform="pandas")),
    ("per", CountryRecoder(country="Peru",          locus=per_locus).set_output(transform="pandas")),
    ("skr", CountryRecoder(country="South Korea",   locus=skr_locus).set_output(transform="pandas")),
    ("bra", CountryRecoder(country="Brazil",        locus=bra_locus).set_output(transform="pandas")),
    ("chi", CountryRecoder(country="Chile",         locus=chi_locus).set_output(transform="pandas")),
    ("ind", CountryRecoder(country="India",         locus=ind_locus).set_output(transform="pandas")),
    ("hun", CountryRecoder(country="Hungary",       locus=hun_locus).set_output(transform="pandas")),
    ("tai", CountryRecoder(country="Taiwan",        locus=tai_locus).set_output(transform="pandas")),
    ("ksa", CountryRecoder(country="Saudi Arabia",  locus=ksa_locus).set_output(transform="pandas")),
    ("por", CountryRecoder(country="Portugal",      locus=por_locus).set_output(transform="pandas")),
    ("swi", CountryRecoder(country="Switzerland",   locus=swi_locus).set_output(transform="pandas")),
    ("egy", CountryRecoder(country="Egypt",         locus=egy_locus).set_output(transform="pandas")),
    ("bru", CountryRecoder(country="Brunei",        locus=bru_locus).set_output(transform="pandas")),
    ("mal", CountryRecoder(country="Mali",          locus=mal_locus).set_output(transform="pandas")),
    ("tha", CountryRecoder(country="Thailand",      locus=tha_locus).set_output(transform="pandas")),
    ("bar", CountryRecoder(country="Barbados",      locus=bar_locus).set_output(transform="pandas")),
    ("cyp", CountryRecoder(country="Cyprus",        locus=cyp_locus).set_output(transform="pandas")),
    ("sin", CountryRecoder(country="Singapore",     locus=sin_locus).set_output(transform="pandas")),
    ("ina", CountryRecoder(country="Indonesia",     locus=ina_locus).set_output(transform="pandas")),
    ("bos", CountryRecoder(country="Bosnia and Herzegovina", locus=bos_locus).set_output(transform="pandas")),
    ("bah", CountryRecoder(country="Bahrain",       locus=bah_locus).set_output(transform="pandas")),
    ("cam", CountryRecoder(country="Cameroon",      locus=cam_locus).set_output(transform="pandas")),
    ("rsa", CountryRecoder(country="South Africa",  locus=rsa_locus).set_output(transform="pandas")),
    ("phi", CountryRecoder(country="Philippines",   locus=phi_locus).set_output(transform="pandas")),
    ("qat", CountryRecoder(country="Qatar",         locus=qat_locus).set_output(transform="pandas")),
    ("ire", CountryRecoder(country="Ireland",       locus=ire_locus).set_output(transform="pandas")),
    ("mex", CountryRecoder(country="Mexico",        locus=mex_locus).set_output(transform="pandas")),
    ("ban", CountryRecoder(country="Bangladesh",    locus=ban_locus).set_output(transform="pandas")),
    ("est", CountryRecoder(country="Estonia",       locus=est_locus).set_output(transform="pandas")),
    ("slk", CountryRecoder(country="Slovakia",      locus=slk_locus).set_output(transform="pandas")),
    ("cz",  CountryRecoder(country="Czech Republic",locus=cz_locus).set_output(transform="pandas")),
    ("gha", CountryRecoder(country="Ghana",         locus=gha_locus).set_output(transform="pandas")),
    ("ice", CountryRecoder(country="Iceland",       locus=ice_locus).set_output(transform="pandas")),
    ("slv", CountryRecoder(country="Slovenia",      locus=slv_locus).set_output(transform="pandas")),
    ("irn", CountryRecoder(country="Iran",          locus=irn_locus).set_output(transform="pandas")),
    ("pak", CountryRecoder(country="Pakistan",      locus=pak_locus).set_output(transform="pandas")),
    ("jam", CountryRecoder(country="Jamaica",       locus=jam_locus).set_output(transform="pandas")),
    ("pol", CountryRecoder(country="Poland",        locus=pol_locus).set_output(transform="pandas")),
])

clean_trans = ColumnTransformer([
    ("cleaner", clean_pipe, ['country'])
],
remainder="passthrough").set_output(transform="pandas")

In [6]:
df_1 = clean_trans.fit_transform(df_all)
df_1.columns = recover_columns_names(df_1)
df_1

,country,ult_affiliation,first_affiliation,Affiliations,PubMed ID,Title,Publication Year,Author,Country
0,China,"Department of Geriatrics, Institute of Geronto...","Department of Geriatrics, Institute of Geronto...","Department of Geriatrics, Institute of Geronto...",38853292,"Physical frailty, genetic predisposition, and ...",2024,Yucong Zhang,China.
1,China,"Key Laboratory of Vascular Aging, Ministry of ...","Key Laboratory of Vascular Aging, Ministry of ...","Key Laboratory of Vascular Aging, Ministry of ...",38853292,"Physical frailty, genetic predisposition, and ...",2024,Yucong Zhang,China.
2,China,"Department of Geriatrics, Institute of Geronto...","Department of Geriatrics, Institute of Geronto...","Department of Geriatrics, Institute of Geronto...",38853292,"Physical frailty, genetic predisposition, and ...",2024,Man Liu,China.
3,China,"Key Laboratory of Vascular Aging, Ministry of ...","Key Laboratory of Vascular Aging, Ministry of ...","Key Laboratory of Vascular Aging, Ministry of ...",38853292,"Physical frailty, genetic predisposition, and ...",2024,Man Liu,China.
4,China,Division of Cardiothoracic and Vascular Surger...,Division of Cardiothoracic and Vascular Surger...,Division of Cardiothoracic and Vascular Surger...,38853292,"Physical frailty, genetic predisposition, and ...",2024,Jiajun Li,China.
...,...,...,...,...,...,...,...,...,...
187548,UK,"St George's Hospital Medical School, and St Ge...","St George's Hospital Medical School, and St Ge...","St George's Hospital Medical School, and St Ge...",12848262,Polymorphic sequence variants in medicine: a c...,2003,Shirley V Hodgson,London. s.hodgson@sghms.ac.uk
187549,UK,"The Lancet, Jamestown Road, NW BY, London, UK","The Lancet, Jamestown Road, NW BY, London, UK","The Lancet, 32 Jamestown Road, NW1 7BY, London...",12767753,UK Biobank: a project in search of a protocol?,2003,Virginia Barbour,UK. virginia.barbour@lancet.com <virginia.barb...
187550,Japan,"Cellbank, Department of Genetics and Mutagenes...","Cellbank, Department of Genetics and Mutagenes...","Cellbank, Department of Genetics and Mutagenes...",12693011,"[Ethical, legal, and social issues of genome r...",2003,Tohru Masui,Japan. masui@nihs.go.jp
187551,UK,"MRC Human Genetics Unit, Western General Hospi...","MRC Human Genetics Unit, Western General Hospi...","MRC Human Genetics Unit, Western General Hospi...",12049178,Gene-environment interactions--the BioBank UK ...,2002,A F Wright,UK. alan.wright@hgu.mrc.ac.uk


In [7]:
test = df_1[~df_1['country'].isnull()]['country'].unique().tolist()
test.sort()
test

['AMRA',
 'Academic',
 'Academic Rheumatology and',
 'African Population and Health Research Center',
 'Airways',
 'Alan',
 'Albania',
 'Albert',
 'Allergy',
 'Alzheimer',
 'Ana Aslan International Foundation',
 'Andrew',
 'Angers',
 'Apia Samoa',
 'Argentina',
 'Armenia',
 'Associate Editor',
 'AstraZeneca',
 'Athens',
 'Aurora',
 'Austin',
 'Australia',
 'Austria',
 'B',
 'BCNAIM',
 'BHF Data Science Centre',
 'BRC Oxford Haematology Theme',
 'BT BA',
 'Bahrain',
 'Bangladesh',
 'Barbados',
 'Basic',
 'Bath',
 'Behavioral',
 'Behavioural',
 'Belarus',
 'Belgium',
 'Bellvitge Biomedical Research Institute',
 'Berkeley',
 'Bethesda',
 'Big',
 'Bilthoven',
 'Biobank Management Center of the TriService General Hospital',
 'Bioimaging',
 'Bioinformatics',
 'Biomedical',
 'Bioscience',
 'Biosciences',
 'Birmingham',
 'Bolivia',
 'Bosnia and Herzegovina',
 'Botswana',
 'Boulder',
 'Brain',
 'Brazil',
 'Breakthrough Breast Cancer Research Centre',
 'Breast',
 'Brigham',
 'Broad',
 'Brownsvil

In [8]:
len(test)

999